<a href="https://colab.research.google.com/github/gmrammohan15/EVA4/blob/master/S8-Assignment/S8_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import sys
sys.path.append('/content/gdrive/My Drive/S8-Assignment/')
!ls /content/gdrive/My\ Drive/S8-Assignment

dataloader.py	    __init__.py       prepare_dataset.py  resnet.py
data_transforms.py  misclassified.py  __pycache__	  stats.py
eval.py		    models	      README.md		  transforms.py


In [0]:
import numpy as np
import data_transforms
from torchvision import transforms

import importlib
importlib.reload(data_transforms)

transformer = data_transforms.Transforms(normalize=True, mean=(0.4914, 0.4822, 0.4465), stdev=(0.247, 0.243, 0.261))

#Train transformations
train_transforms = transformer.train_transforms([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32,padding=4),
    #transforms.RandomErasing(value = tuple((0.4914, 0.4822, 0.4465)))
])

# Test  transformations
test_transforms = transformer.test_transforms()

In [4]:
from torchvision import datasets
train_data = datasets.CIFAR10('./data', train=True, download=True, transform=train_transforms)
test_data = datasets.CIFAR10('./data', train=False, download=True, transform=test_transforms)


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [0]:
import dataloader

import importlib
importlib.reload(dataloader)

dl = dataloader.DataLoader(batch_size=128)

# train dataloader
train_loader = dl.load(train_data)

# test dataloader
test_loader = dl.load(test_data)

In [0]:
import matplotlib.pyplot as plt
import torchvision
import numpy as np

# functions to show an image
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

# get some random training images
dataiter = iter(train_loader)
images, labels  = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(32)))

In [6]:
#from models import resnet
import resnet

import torch

import importlib
importlib.reload(resnet)
#importlib.reload(eval)


model = resnet.ResNet18()
use_cuda = torch.cuda.is_available()
model.device = torch.device("cuda" if use_cuda else "cpu")
model.to(model.device)
model.summary((3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 16, 16]          73,728
      BatchNorm2d-14          [-1, 128,

In [7]:
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import OneCycleLR
from torch.optim.lr_scheduler import StepLR


EPOCHS = 10
l2_loss = 0.0001
l1_lambda = 0
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=l2_loss)

#scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
schedular = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.5, steps_per_epoch=len(train_loader), epochs=EPOCHS)


model.evaluate(optimizer, train_loader, test_loader, EPOCHS, criterion, schedular, True, l1_lambda)

Epoch 0, Train Accuracy: 0.4168799817562103 , Test Accuracy: 0.5277999639511108
Epoch 1, Train Accuracy: 0.5979399681091309 , Test Accuracy: 0.5974000096321106
Epoch 2, Train Accuracy: 0.6971799731254578 , Test Accuracy: 0.6953999996185303
Epoch 3, Train Accuracy: 0.7548999786376953 , Test Accuracy: 0.7638999819755554
Epoch 4, Train Accuracy: 0.7929199934005737 , Test Accuracy: 0.7723999619483948
Epoch 5, Train Accuracy: 0.8172599673271179 , Test Accuracy: 0.8129000067710876
Epoch 6, Train Accuracy: 0.8339799642562866 , Test Accuracy: 0.8325999975204468
Epoch 7, Train Accuracy: 0.8528599739074707 , Test Accuracy: 0.8306999802589417
Epoch 8, Train Accuracy: 0.865559995174408 , Test Accuracy: 0.8420000076293945
Epoch 9, Train Accuracy: 0.875499963760376 , Test Accuracy: 0.8554999828338623
